In [1]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [ ]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [2]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-03-25 20:44:17.069210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-25 20:44:17.069253: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [65]:

nActions = 4
tableSize = 4
hiddenFeatures = 16
# hiddenFeatures = 32
# activation="relu"
activation="leaky_relu"

# Higher temporal window confuses the model
# temporalWindow = 5
temporalWindow = 1



env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))

# Result look alike
model.add(Activation(activation=activation))
# model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha

# Worst result
# model.add(Activation(activation="elu")) # Tune alpha


# Additional layers
model.add(Dense(hiddenFeatures))
model.add(Activation(activation=activation))

model.add(Dense(hiddenFeatures))
model.add(Activation(activation=activation))


# Output layer
model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()

# Less invalid moves with more limit
replayMemory = SequentialMemory(limit=2000, window_length=temporalWindow)

dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=1000,
    target_model_update=0.01,
    policy=policy
)
dqn.compile(optimizer=Adam(lr=0.001), metrics=["mae"])
dqn.fit(env=env, nb_steps=20000, visualize=False, verbose=2)


/usr/local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 20000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   686/20000: episode: 1, duration: 1.145s, episode steps: 686, steps per second: 599, episode reward: -27436.000, mean reward: -39.994 [-50.000, 160.000], mean action: 1.067 [0.000, 3.000],  loss: --, mae: --, mean_q: --


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  1058/20000: episode: 2, duration: 2.520s, episode steps: 372, steps per second: 148, episode reward: -13456.000, mean reward: -36.172 [-50.000, 76.000], mean action: 1.110 [0.000, 3.000],  loss: 938.493001, mae: 11.008136, mean_q: 0.757472
  1473/20000: episode: 3, duration: 2.504s, episode steps: 415, steps per second: 166, episode reward: -12560.000, mean reward: -30.265 [-50.000, 136.000], mean action: 1.610 [0.000, 3.000],  loss: 339.907043, mae: 27.749821, mean_q: -25.188963
  1756/20000: episode: 4, duration: 1.686s, episode steps: 283, steps per second: 168, episode reward: -5934.000, mean reward: -20.968 [-50.000, 80.000], mean action: 1.035 [0.000, 3.000],  loss: 341.804840, mae: 77.952507, mean_q: -92.816711
  2045/20000: episode: 5, duration: 1.762s, episode steps: 289, steps per second: 164, episode reward: -7488.000, mean reward: -25.910 [-50.000, 160.000], mean action: 1.913 [0.000, 3.000],  loss: 393.496429, mae: 116.197121, mean_q: -141.725479
  2391/20000: episode: 6

# Simulate Game

In [67]:
# Play the game with predicts

def log2_transform(grid):
    transformed = np.where(grid > 0, np.log2(grid), 0)
    return transformed

actionMapping = {
    0: "up",
    1: "down",
    2: "left",
    3: "right"
}

games = [Game2048() for _ in range(100)]
metrics = {
    "gameId": [],
    "score": [],
    "higherTile": [],
    "invalidMove (%)": [],
    "steps": [],
    "upMoves": [],
    "downMoves": [],
    "leftMoves": [],
    "rightMoves": [],
    "upSeqAvg": [],
    "downSeqAvg": [],
    "leftSeqAvg": [],
    "rightSeqAvg": [],
}
for i, game in enumerate(games):
    # Metrics
    steps = 0
    invalidMove = 0
    upMoves = 0
    downMoves = 0
    leftMoves = 0
    rightMoves = 0
    upSeqAvg = 0
    downSeqAvg = 0
    leftSeqAvg = 0
    rightSeqAvg = 0

    sequences = {
        "up": [],
        "down": [],
        "left": [],
        "right": []
    }
    lastMove = None

    # print(f"Game {i + 1} initial state:")
    # game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)[0]
        observation = log2_transform(observation)
        predictedAction = dqn.forward(observation)
        
        # print(f"\nPredicted Action: move {actionMapping[predictedAction]}")
        if not game.move(actionMapping[predictedAction]):
            invalidMove += 1
        
        # print(f"Game {i + 1} currrent state:")
        # game.render()

        # input("\nPress any key to continue...")

        steps += 1
        if actionMapping[predictedAction] == "up":
            upMoves += 1
            if lastMove == "up":
                sequences["up"][-1] += 1
            else:
                sequences["up"].append(1)
        elif actionMapping[predictedAction] == "down":
            downMoves += 1
            if lastMove == "down":
                sequences["down"][-1] += 1
            else:
                sequences["down"].append(1)
        elif actionMapping[predictedAction] == "left":
            leftMoves += 1
            if lastMove == "left":
                sequences["left"][-1] += 1
            else:
                sequences["left"].append(1)
        elif actionMapping[predictedAction] == "right":
            rightMoves += 1
            if lastMove == "right":
                sequences["right"][-1] += 1
            else:
                sequences["right"].append(1)
        
        lastMove = actionMapping[predictedAction]

    print(f"Game {i + 1} is over.")

    metrics["gameId"].append(i)
    metrics["score"].append(game.score)
    metrics["higherTile"].append(np.max(game.grid))
    metrics["invalidMove (%)"].append(invalidMove/steps)
    metrics["steps"].append(steps)
    metrics["upMoves"].append(upMoves)
    metrics["downMoves"].append(downMoves)
    metrics["leftMoves"].append(leftMoves)
    metrics["rightMoves"].append(rightMoves)
    metrics["upSeqAvg"].append(np.mean(np.array(sequences["up"])))
    metrics["downSeqAvg"].append(np.mean(np.array(sequences["down"])))
    metrics["leftSeqAvg"].append(np.mean(np.array(sequences["left"])))
    metrics["rightSeqAvg"].append(np.mean(np.array(sequences["right"])))


metrics = pd.DataFrame(metrics).set_index("gameId")
display(metrics)
display(metrics.describe())


/tmp/ipykernel_5981/802290428.py:4: RuntimeWarning: divide by zero encountered in log2
  transformed = np.where(grid > 0, np.log2(grid), 0)


Game 1 is over.
Game 2 is over.
Game 3 is over.
Game 4 is over.
Game 5 is over.
Game 6 is over.
Game 7 is over.
Game 8 is over.
Game 9 is over.
Game 10 is over.
Game 11 is over.
Game 12 is over.
Game 13 is over.
Game 14 is over.
Game 15 is over.
Game 16 is over.
Game 17 is over.
Game 18 is over.
Game 19 is over.
Game 20 is over.
Game 21 is over.
Game 22 is over.
Game 23 is over.
Game 24 is over.
Game 25 is over.
Game 26 is over.
Game 27 is over.
Game 28 is over.
Game 29 is over.
Game 30 is over.
Game 31 is over.
Game 32 is over.
Game 33 is over.
Game 34 is over.
Game 35 is over.
Game 36 is over.
Game 37 is over.
Game 38 is over.
Game 39 is over.
Game 40 is over.
Game 41 is over.
Game 42 is over.
Game 43 is over.
Game 44 is over.
Game 45 is over.
Game 46 is over.
Game 47 is over.
Game 48 is over.
Game 49 is over.
Game 50 is over.
Game 51 is over.
Game 52 is over.
Game 53 is over.
Game 54 is over.
Game 55 is over.
Game 56 is over.
Game 57 is over.
Game 58 is over.
Game 59 is over.
Game 6

,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
gameId,,,,,,,,,,,,
0,1432,128,0.461818,275,91,94,59,31,2.333333,2.764706,1.594595,1.068966
1,996,64,0.514403,243,30,89,95,29,1.034483,3.296296,2.638889,1.035714
2,628,64,0.350746,134,17,64,29,24,1.000000,3.764706,1.074074,1.000000
3,716,64,0.631356,236,18,145,50,23,1.058824,6.904762,1.785714,1.000000
4,512,64,0.489655,145,17,85,23,20,1.062500,4.473684,1.045455,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1000,128,0.226950,141,24,62,35,20,1.000000,2.066667,1.129032,1.000000
96,700,64,0.384106,151,11,73,37,30,1.000000,5.214286,1.057143,1.071429
97,784,64,0.550000,220,27,138,32,23,1.000000,5.111111,1.103448,1.045455


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,674.320000,67.520000,0.353450,155.690000,28.910000,69.050000,36.530000,21.200000,1.343617,2.928931,1.443094,1.047233
std,305.846165,30.728686,0.199640,80.968343,24.533257,57.679255,25.404388,9.772152,0.709016,1.753957,0.722043,0.167164
min,196.000000,16.000000,0.000000,45.000000,8.000000,9.000000,8.000000,7.000000,1.000000,1.000000,1.000000,1.000000
25%,511.000000,64.000000,0.238378,96.750000,16.000000,29.750000,21.000000,16.750000,1.000000,1.433333,1.032576,1.000000
50%,630.000000,64.000000,0.369817,132.000000,20.000000,50.000000,28.500000,20.000000,1.055556,2.527536,1.090909,1.000000
75%,758.000000,64.000000,0.499300,203.250000,30.250000,89.000000,38.250000,24.250000,1.157143,3.811538,1.563122,1.045996
max,1780.000000,128.000000,0.754717,373.000000,153.000000,305.000000,138.000000,94.000000,4.781250,8.026316,4.125000,2.611111


# Manual Game

In [ ]:
moves = ["up", "left", "right", "down"]
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


